In [9]:
!pip install pandas numpy seaborn matplotlib scikit-learn --quiet

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Enable inline plotting (use this only in Jupyter notebooks)
%matplotlib inline

# Set Seaborn style for plots
sns.set(style="whitegrid")

print("Libraries loaded successfully!")


Libraries loaded successfully!


In [14]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google'

In [ ]:
import zipfile

zip_path = "archive.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("data")

print("ZIP extracted to 'data/'")

In [ ]:
import os

for root, dirs, files in os.walk("data"):
    for file in files:
        print(os.path.join(root, file))

In [ ]:
import os

for root, dirs, files in os.walk("data"):
    for file in files:
        print(os.path.join(root, file))

In [ ]:
import pandas as pd

df = pd.read_csv("data/train_0irEZ2H.csv")

df.info()
df.head()

In [ ]:
import pandas as pd

df = pd.read_csv("data/train_0irEZ2H.csv")

categorical_fields = df.select_dtypes(include=['object', 'datetime']).columns.tolist()
numerical_fields = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("categorical.fields")
print(categorical_fields)

In [ ]:
print("\nnumerical.fields")
print(numerical_fields)

In [ ]:
print("\nmissing_values")
print(df[numerical_fields].isnull().sum())

df[numerical_fields] = df[numerical_fields].fillna(df[numerical_fields].mean())

print("\nnumerical.impute() done (mean imputation)")

In [ ]:
import pandas as pd

df = pd.read_csv("data/train_0irEZ2H.csv")

df['key'] = df['week'].astype(str) + '_' + df['store_id'].astype(str)

df = df.drop(['record_ID', 'week', 'store_id', 'sku_id', 'total_price',
              'base_price', 'is_featured_sku', 'is_display_sku'], axis=1)

df = df.groupby('key').sum()

print(df.head())
print("\nShape of final grouped dataset:", df.shape)

In [ ]:
import matplotlib.pyplot as plt

df[:100].plot(figsize=(12, 8))

plt.title("Units Sold Over Time (First 100 Records)", fontsize=16)
plt.xlabel("Key (Week_Store)", fontsize=12)
plt.ylabel("Units Sold", fontsize=12)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
df['day_1'] = df['units_sold'].shift(1)
df['day_2'] = df['units_sold'].shift(2)
df['day_3'] = df['units_sold'].shift(3)
df['day_4'] = df['units_sold'].shift(4)

df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("data/train_0irEZ2H.csv")

df['key'] = df['week'].astype(str) + '_' + df['store_id'].astype(str)

df = df.drop(['record_ID', 'week', 'store_id', 'sku_id', 'total_price', 'base_price', 'is_featured_sku', 'is_display_sku'], axis=1)

df = df.groupby('key').sum()

df['day_1'] = df['units_sold'].shift(1)
df['day_2'] = df['units_sold'].shift(2)
df['day_3'] = df['units_sold'].shift(3)
df['day_4'] = df['units_sold'].shift(4)

In [ ]:
df = df.dropna()

df[:100].plot(figsize=(12, 8))
plt.xlabel("key")
plt.ylabel("units_sold and lag values")
plt.title("Sales Trends with Lag Features")
plt.grid(True)
plt.show()

In [ ]:
import numpy as np

x1, x2, x3, x4, y = df['day_1'], df['day_2'], df['day_3'], df['day_4'], df['units_sold']

x1, x2, x3, x4, y = (
    np.array(x1).reshape(-1, 1),
    np.array(x2).reshape(-1, 1),
    np.array(x3).reshape(-1, 1),
    np.array(x4).reshape(-1, 1),
    np.array(y).reshape(-1, 1)
)

x = np.concatenate((x1, x2, x3, x4), axis=1)

split_percentage = 15
test_split = int(len(df) * (split_percentage / 100))

x_train, x_test = x[:-test_split], x[-test_split:]
y_train, y_test = y[:-test_split], y[-test_split:]

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_regressor = RandomForestRegressor()

rf_regressor.fit(x_train, y_train.ravel())

y_pred = rf_regressor.predict(x_test)

print("R Sq. Score for Random Forest Regression :", rf_regressor.score(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 8)

plt.plot(y_pred[-100:], label='Predictions')
plt.plot(y_test[-100:], label='Actual Sales')

plt.legend(loc='upper left')

plt.show()

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor

xgb_regressor = XGBRegressor()
xgb_regressor.fit(x_train, y_train.ravel())

y_pred = xgb_regressor.predict(x_test)

print("R Sq. Score for XGBoost:", xgb_regressor.score(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 6)

plt.plot(y_pred[-100:], label='Predictions', color='orange')
plt.plot(y_test[-100:], label='Actual Sales', color='blue')

plt.legend(loc="upper left")
plt.title("XGBoost Predictions vs Actual Sales (Last 100 Days)")
plt.xlabel("Sample")
plt.ylabel("Units Sold")
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start=50, stop=250, num=10)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(0, 120, num=20)]
max_depth.append(None)

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]

random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

from pprint import pprint
pprint(random_grid)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=10,
    cv=3,
    verbose=2,
    random_state=0,
    n_jobs=-1
)

rf_random.fit(x_train, y_train.ravel())

In [ ]:
print("Best Hyperparameters:\n", rf_random.best_params_)

best_random = rf_random.best_estimator_
y_pred = best_random.predict(x_test)

r2 = best_random.score(x_test, y_test)
print("R² Score for Random Forest Regression:", r2)

n = len(y_test)
k = x_test.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)
print("Adjusted R² Score for Random Forest Regression:", adj_r2)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (30, 8)

plt.plot(y_pred[500:800], label='Predictions')
plt.plot(y_test[500:800], label='Actual Sales')

plt.legend(loc="upper left")

plt.savefig('final.png')

plt.show()

In [ ]:
import pickle
import numpy as np

pickle.dump(rf_regressor, open('sales_demand_forecasting.pkl', 'wb'))

features = np.array([[682.0, 535.0, 210.0, 782.0]])

print("Predicted Sales Units:", rf_regressor.predict(features)[0])